# See the quadrant 3 fitsfile to reduce

In [8]:
from astropy.io import fits
import numpy as np
import glob
import itertools
import re
from astropy.coordinates import SkyCoord
from astropy import units as u

In [9]:
def flatten(listOfLists):
    "Flatten one level of nesting"
    return itertools.chain.from_iterable(listOfLists)

In [10]:
grismnumber = 3
datadir = '../data/grism{}/'.format(grismnumber)

files = glob.glob(datadir+'*.fits')

lista = []
screenflats = []
arcspectrums = []
quadlist =[]
objectlist = []

for file in files:
    tmpfits = fits.open(file)
    tmpobject = tmpfits[0].header['OBJECT']
    tmpcat = tmpfits[0].header['HIERARCH ESO DPR CATG']
    tmptype = tmpfits[0].header['HIERARCH ESO DPR TYPE']
    tmptech = tmpfits[0].header['HIERARCH ESO DPR TECH']
    tmpinsmode = tmpfits[0].header["HIERARCH ESO INS MODE"]
    quad = tmpfits[0].header['HIERARCH ESO OCS CON QUAD']

    lista.append((tmptype,quad))

    if tmptype == 'FLAT,LAMP' and quad == grismnumber:
        screenflats.append((file,'IFU_SCREEN_FLAT',quad))
    if tmptype == 'WAVE,LAMP' and quad == grismnumber:
        arcspectrums.append((file,'IFU_ARC_SPECTRUM',quad))
    if tmptype == 'OBJECT' and quad == grismnumber:
        objectlist.append((file,'IFU_SCIENCE',quad))


## Move each object to its on folder then process them there

In [11]:
objectfolder = datadir+'objects'
for ob in objectlist:
    date = ob[0].split('/')[-1].split('.')[1]
    pathob = '{}/{}'.format(objectfolder,date)
    if not os.path.exists(pathob):
        os.mkdir(pathob)
    os.system('cp {} {}'.format(ob[0],pathob))
    infofile = "{}/{}".format(pathob,'info.txt')
    with open(infofile,'w') as f:
        temp = fits.open(ob[0])
        fitheader = temp[0].header
        date = fitheader['DATE-OBS']
        exptime = fitheader['EXPTIME']
        ra = fitheader['RA']
        ra2temp = str(fitheader[11:12])
        ra2 = re.search('(\d+)\:(\d+)\:(\d+.\d)',ra2temp)[0]
        ifura = fitheader['HIERARCH ESO INS IFU RA']
        ifudec = fitheader['HIERARCH ESO INS IFU DEC']
        c_icrs = SkyCoord(ra=ifura*u.degree, dec=ifudec*u.degree, frame='icrs')
        ifucoord = c_icrs.to_string('hmsdms').split(' ')
        ifura2 = ifucoord[0]
        ifudec2 = ifucoord[1]
        dec = fitheader['DEC']
        dec2temp = str(fitheader[12:13])
        dec2 = re.search('(\d+)\:(\d+)\:(\d+.\d)',dec2temp)[0]
        name = fitheader['HIERARCH ESO OBS TARG NAME']
        f.write('name: {} \ndate: {}\nexptime: {}\nra: {} \nra2 {} \ndec: {} \ndec2 {} \nifuRA {} \nifuDEC {}\n'.format(name,date,exptime,ra,ra2,dec,dec2,ifura2,ifudec2))

#  Flat, Object and Wave

In [12]:
ob = [ i[0] for i in lista]
np.unique(ob,return_counts=True)

(array(['FLAT,LAMP', 'OBJECT', 'WAVE,LAMP'],
       dtype='<U9'), array([10, 31, 16]))

So few obsevation of the same thing. Fist at of monght 1-4, and they overlap 3 and 4

# Bias

### Also there are bias in cal

The keyword ESO INS MODE is set accordingly to ’IMG’ for direct
imaging frames, and to ’MOS’ for any calibration associated to spectroscopy (either MOS or IFU), to indicate the intended use for the data.

# DO category: BIAS

**DPR TYPE = BIAS**

**DPR TECH = IMAGE**

In [28]:
biasdir = '../data/grism{}/bias/*.fits'.format(grismnumber)
filesbias = glob.glob(biasdir)



listinsmode = []
listaobdate = [] 
listname = []

for file in filesbias:
    tmpfits = fits.open(file)
    tmpdate = tmpfits[0].header['HIERARCH ESO OBS START']
    tmpobject = tmpfits[0].header['OBJECT']
    tmpcat = tmpfits[0].header['HIERARCH ESO DPR CATG']
    tmptype = tmpfits[0].header['HIERARCH ESO DPR TYPE']
    tmptech = tmpfits[0].header['HIERARCH ESO DPR TECH']
    tmpinsmode = tmpfits[0].header["HIERARCH ESO INS MODE"]    
    quad = tmpfits[0].header['HIERARCH ESO OCS CON QUAD']


    listinsmode.append((file,tmpinsmode,quad))
    listaobdate.append((tmptype,tmpdate,quad))


Only interested in MOS so delte the IMG

In [29]:
set([i[1:3] for i in listinsmode])

{('MOS', 1)}

In [30]:
for i,j,k in listinsmode:
    if j == 'IMG' and os.path.exists(i):
        os.remove(i)   
    if k != 1:
        os.remove(i)


In [31]:
len(listinsmode)

15

# Calibrations files

We find the LINE_CATALOG, and IFU_IDENT. 

In [32]:
caldir = '../esorex/calibrations/calib/vimos-3.2.3/cal/*'
filescal = glob.glob(caldir)

keyw =[ ]

for file in filescal:
    #print(file)
    tmpfits = fits.open(file)
    #name = file.split('/')[-1]
    name = file
    #tmpdate = tmpfits[0].header['HIERARCH ESO OBS START']
    #tmpobject = tmpfits[0].header['OBJECT']
    tmpcat = tmpfits[0].header['HIERARCH ESO PRO CATG']
    tmptype = 'no'
    try:
        tmptype = tmpfits[0].header['HIERARCH ESO PRO TYPE']
        #tmptech = tmpfits[0].header['HIERARCH ESO DPR TECH']
        #tmpinsmode = tmpfits[0].header["HIERARCH ESO INS MODE"]
    except:
        b='a'
    keyw.append((name,tmpcat,tmptype))
keyw

[('../esorex/calibrations/calib/vimos-3.2.3/cal/grs_HR_redh.3.tfits',
  'GRISM_TABLE',
  'no'),
 ('../esorex/calibrations/calib/vimos-3.2.3/cal/VIMOS_GRS_MR_OS_red.fits',
  'CONFIG_TABLE',
  'no'),
 ('../esorex/calibrations/calib/vimos-3.2.3/cal/g158_100.tfits',
  'STD_FLUX_TABLE',
  'no'),
 ('../esorex/calibrations/calib/vimos-3.2.3/cal/grs_LR_red.2.tfits',
  'GRISM_TABLE',
  'no'),
 ('../esorex/calibrations/calib/vimos-3.2.3/cal/hilt600.tfits',
  'STD_FLUX_TABLE',
  'no'),
 ('../esorex/calibrations/calib/vimos-3.2.3/cal/badpixel.8.tfits',
  'CCD_TABLE',
  'REDUCED'),
 ('../esorex/calibrations/calib/vimos-3.2.3/cal/grs_HR_blue.2.tfits',
  'GRISM_TABLE',
  'no'),
 ('../esorex/calibrations/calib/vimos-3.2.3/cal/lcat_HR_blue.tfits',
  'LINE_CATALOG',
  'no'),
 ('../esorex/calibrations/calib/vimos-3.2.3/cal/ngc7293.tfits',
  'STD_FLUX_TABLE',
  'no'),
 ('../esorex/calibrations/calib/vimos-3.2.3/cal/grs_HR_orange.4.tfits',
  'GRISM_TABLE',
  'no'),
 ('../esorex/calibrations/calib/vimos-3.2

# LINE CATALOG

In [33]:
gri = 'HR_blue'
lcatname = 'lcat_{}.tfits'.format(gri)
linecat = [(i[0],'LINE_CATALOG') for i in keyw if i[1] == 'LINE_CATALOG' and i[0].split('/')[-1] == lcatname ]
linecat 

[('../esorex/calibrations/calib/vimos-3.2.3/cal/lcat_HR_blue.tfits',
  'LINE_CATALOG')]

## Find the IFU_INDEX

ifu_ident_grism_q.fits (where q indicates the VIMOS quadrant number, and grism the grism
name). 

A new set of fiber identification files was added in 2006, in order to support IFU data obtained around
2006 and after. The earlier IFU_IDENT files had the suffix "_2006" added to their names. A further set was
added after the Summer 2010 intervention, and similarly the previous IFU_IDENT files had the suffix "_2010"
added to their names.

**So need to downlaod the olde ones**

In [34]:
[i for i in keyw if i[1] == 'IFU_IDENT'and 'HR_blue' in i[0].split('/')[-1] ]

[('../esorex/calibrations/calib/vimos-3.2.3/cal/ifu_ident_HR_blue.2.fits',
  'IFU_IDENT',
  'no'),
 ('../esorex/calibrations/calib/vimos-3.2.3/cal/ifu_ident_HR_blue_2010.1.fits',
  'IFU_IDENT',
  'no'),
 ('../esorex/calibrations/calib/vimos-3.2.3/cal/ifu_ident_HR_blue_2010.4.fits',
  'IFU_IDENT',
  'no'),
 ('../esorex/calibrations/calib/vimos-3.2.3/cal/ifu_ident_HR_blue_2011.3.fits',
  'IFU_IDENT',
  'no'),
 ('../esorex/calibrations/calib/vimos-3.2.3/cal/ifu_ident_HR_blue.3.fits',
  'IFU_IDENT',
  'no'),
 ('../esorex/calibrations/calib/vimos-3.2.3/cal/ifu_ident_HR_blue.1.fits',
  'IFU_IDENT',
  'no'),
 ('../esorex/calibrations/calib/vimos-3.2.3/cal/ifu_ident_HR_blue.4.fits',
  'IFU_IDENT',
  'no'),
 ('../esorex/calibrations/calib/vimos-3.2.3/cal/ifu_ident_HR_blue_2010.3.fits',
  'IFU_IDENT',
  'no'),
 ('../esorex/calibrations/calib/vimos-3.2.3/cal/ifu_ident_HR_blue_2011.2.fits',
  'IFU_IDENT',
  'no'),
 ('../esorex/calibrations/calib/vimos-3.2.3/cal/ifu_ident_HR_blue_2010.2.fits',
  'I

In [35]:
quadrant =1 
ifuidentname = 'ifu_ident_{}.{}.fits'.format(gri,quadrant)
ifuident = [(i[0],'IFU_IDENT') for i in keyw if i[1] == 'IFU_IDENT' and i[0].split('/')[-1] == ifuidentname ]
ifuident

[('../esorex/calibrations/calib/vimos-3.2.3/cal/ifu_ident_HR_blue.1.fits',
  'IFU_IDENT')]

# One Standard Star Exposure

**DO category: IFU_STANDARD**

**DPR TYPE = STD**

**DPR TECH = IFU**

Downlaed from archive usng the script stdjune2.sh


We want:

- **HR_blue**
- **Quadrant one**

So delte the rest.

In [36]:
grismnumber = 1

stddir = '../data/grism{}/STD/*.fits'.format(grismnumber)
filesstd = glob.glob(stddir)

keystd = []


for file in filesstd:
    #print(file)
    tmpfits = fits.open(file)
    name = file
    #tmpdate = tmpfits[0].header['HIERARCH ESO OBS START']
    #tmpobject = tmpfits[0].header['OBJECT']
    tmptech = tmpfits[0].header['HIERARCH ESO DPR TECH']
    tmpquad = tmpfits[0].header['HIERARCH ESO OCS CON QUAD']
    tmpgrism = tmpfits[0].header['HIERARCH ESO INS GRIS{} NAME'.format(tmpquad)]
    tmpcat = tmpfits[0].header['HIERARCH ESO DPR CATG']
    keystd.append((name, tmpquad, tmpgrism, tmptech))

In [37]:
for i, (*j) in keystd:
    if j != [1,gri,'IFU']:
        os.remove(i)
set(keystd)

{('../data/grism1/STD/VIMOS.2003-06-03T09:37:33.630.fits',
  1,
  'HR_blue',
  'IFU'),
 ('../data/grism1/STD/VIMOS.2003-06-03T09:44:10.439.fits',
  1,
  'HR_blue',
  'IFU')}

In [38]:
stdlist = [(i[0],'IFU_STANDARD') for i in keystd ]
stdlist

[('../data/grism1/STD/VIMOS.2003-06-03T09:44:10.439.fits', 'IFU_STANDARD'),
 ('../data/grism1/STD/VIMOS.2003-06-03T09:37:33.630.fits', 'IFU_STANDARD')]

# STD Flux Table

A set of standard star flux tables, corresponding to the 30 spectro-photometric standard stars that are included
in the VIMOS calibration plan

In [39]:
stdtable = [(i[0],'STD_FLUX_TABLE') for i in keyw if i[1] == 'STD_FLUX_TABLE']
stdtable

[('../esorex/calibrations/calib/vimos-3.2.3/cal/g158_100.tfits',
  'STD_FLUX_TABLE'),
 ('../esorex/calibrations/calib/vimos-3.2.3/cal/hilt600.tfits',
  'STD_FLUX_TABLE'),
 ('../esorex/calibrations/calib/vimos-3.2.3/cal/ngc7293.tfits',
  'STD_FLUX_TABLE'),
 ('../esorex/calibrations/calib/vimos-3.2.3/cal/gd50.tfits', 'STD_FLUX_TABLE'),
 ('../esorex/calibrations/calib/vimos-3.2.3/cal/hz44.tfits', 'STD_FLUX_TABLE'),
 ('../esorex/calibrations/calib/vimos-3.2.3/cal/ltt7987.tfits',
  'STD_FLUX_TABLE'),
 ('../esorex/calibrations/calib/vimos-3.2.3/cal/cd32d9927.tfits',
  'STD_FLUX_TABLE'),
 ('../esorex/calibrations/calib/vimos-3.2.3/cal/lds749b.tfits',
  'STD_FLUX_TABLE'),
 ('../esorex/calibrations/calib/vimos-3.2.3/cal/eg274.tfits',
  'STD_FLUX_TABLE'),
 ('../esorex/calibrations/calib/vimos-3.2.3/cal/ltt1788.tfits',
  'STD_FLUX_TABLE'),
 ('../esorex/calibrations/calib/vimos-3.2.3/cal/ltt1020.tfits',
  'STD_FLUX_TABLE'),
 ('../esorex/calibrations/calib/vimos-3.2.3/cal/gd108.tfits',
  'STD_FLUX_

# Extinction Table

In [40]:
extable = [(i[0],'EXTINCT_TABLE') for i in keyw if i[1] == 'EXTINCT_TABLE']
extable

[('../esorex/calibrations/calib/vimos-3.2.3/cal/extinct_table.tfits',
  'EXTINCT_TABLE')]

# Reduction

There are 32 science objects, 2 STD, 50 bias, 2 Flat,Lamp and 15 Wave,Lamp


Step 1:

- [ ] Creat Master bias with vmbias

In [41]:
productfolder = datadir+'products'

# BIAS

It needs BIAS and Bad Pixel table is not required

## Create bias.sof

In [65]:
limit = 5

filename = datadir+'bias.sof'
with open(filename,'w') as file:
    for bias in filesbias[0:limit]:
        cat = 'BIAS'
        string = '{} {}\n'.format(bias,cat)
        file.write(string)

In [75]:
! cat {filename}

../data/grism3/bias/VIMOS.2003-06-03T11:26:20.261.fits BIAS
../data/grism3/bias/VIMOS.2003-06-02T19:30:37.451.fits BIAS
../data/grism3/bias/VIMOS.2003-06-02T19:28:51.438.fits BIAS
../data/grism3/bias/VIMOS.2003-06-02T19:29:44.361.fits BIAS
../data/grism3/bias/VIMOS.2003-06-02T12:36:21.588.fits BIAS


In [76]:
stackmethod = '--StackMethod=Average'
esorexpath = '../esorex/bin/esorex'
routine = 'vmbias'

In [77]:
%%bash -s "{esorexpath}" "{routine}" "{stackmethod}" "{filename}"
$1 $2 $3 $4


     ***** ESO Recipe Execution Tool, version 3.13.1  *****

[ INFO  ] vmbias: Combining 5 frames with method 'Average'
[ INFO  ] vmbias: Computing QC1 parameters...
[ INFO  ] vmbias: Saving MASTER_BIAS image to disk...
[ INFO  ] esorex: Calculating product checksums
[ INFO  ] esorex: Created product master_bias.fits (in place)
[ INFO  ] esorex: 1 product created
[ INFO  ] esorex: Recipe operation(s) took           11.4 seconds to complete.
[ INFO  ] esorex: Total size of 5 raw input frames  =    88.08 MB

[ INFO  ] esorex: => processing rate of     7.75 MB/sec 



In [80]:
mbias = 'master_bias.fits'
masterbias = [(productfolder+'/'+mbias,'MASTER_BIAS')]
products = [mbias]

for p in products:
    copycommand = "mv {} {}".format(p,productfolder)
    os.system(copycommand)

In [81]:
! ls {productfolder}

master_bias.fits


In [82]:
productfolder

'../data/grism3/products'

# Flat and Ac
## vmifucalib

IFU_SCREEN_FLAT | Raw frame | Flat field exposure **Required**

IFU_ARC_SPECTRUM | Raw frame | Arc lamp exposure

MASTER_BIAS | Calibration | Master bias **Required**

LINE_CATALOG | Calibration | Line catalog

IFU_IDENT | Calibration | Fiber identification

CCD_TABLE | Calibration | Bad pixel table

In [83]:
inputfiles = [masterbias,linecat,ifuident,arcspectrums[0:1],screenflats]
inputfiles = list(flatten(inputfiles))

filenamecalib = datadir+'calib.sof'

with open(filenamecalib,'w') as file:
    for sof in inputfiles:
        string = '{} {}\n'.format(sof[0],sof[1])
        file.write(string)

In [84]:
! cat {filenamecalib}

../data/grism3/products/master_bias.fits MASTER_BIAS
../esorex/calibrations/calib/vimos-3.2.3/cal/lcat_HR_blue.tfits LINE_CATALOG
../esorex/calibrations/calib/vimos-3.2.3/cal/ifu_ident_HR_blue.1.fits IFU_IDENT
../data/grism3/VIMOS.2003-06-24T05:41:51.425.fits IFU_ARC_SPECTRUM
../data/grism3/VIMOS.2003-06-04T05:32:48.145.fits IFU_SCREEN_FLAT
../data/grism3/VIMOS.2003-06-04T06:24:37.708.fits IFU_SCREEN_FLAT
../data/grism3/VIMOS.2003-06-04T04:34:46.145.fits IFU_SCREEN_FLAT
../data/grism3/VIMOS.2003-06-06T07:13:49.718.fits IFU_SCREEN_FLAT
../data/grism3/VIMOS.2003-06-04T06:49:18.387.fits IFU_SCREEN_FLAT
../data/grism3/VIMOS.2003-06-06T07:08:16.804.fits IFU_SCREEN_FLAT
../data/grism3/VIMOS.2003-06-06T07:11:40.954.fits IFU_SCREEN_FLAT


In [85]:
esorexpath = '../esorex/bin/esorex'
routine = 'vmifucalib'


In [86]:
%%bash -s "{esorexpath}" "{routine}"  "{filenamecalib}"
$1 $2 $3 


     ***** ESO Recipe Execution Tool, version 3.13.1  *****

[ INFO  ] vmifucalib: *****************************************
[ INFO  ] vmifucalib: Welcome to VIMOS Pipeline release 3.2.3
[ INFO  ] vmifucalib: *****************************************
[ INFO  ] vmifucalib: Loading the input frames...
[ INFO  ] vmifucalib: Processing data from quadrant 3, HR_blue/Free, shutter ON...
[ INFO  ] vmifucalib: Bias removal...
[ INFO  ] vmifucalib: Combining 7 flat fields with method 'MinMax'
[ INFO  ] vmifucalib: Processing spectra in pseudo-slit 2:
[ INFO  ] vmifucalib:   Identify and trace flat field spectra...
[ INFO  ] vmifucalib:   Cross-correlation offset with reference identification: -5.350250
[ INFO  ] vmifucalib:     278 out of 400 spectra were identified and traced.
[ INFO  ] vmifucalib:   Extract flat field spectra...
[ INFO  ] vmifucalib:   Extract arc lamp spectra...
[ INFO  ] vmifucalib:   Compute the wavelength calibration (blind)...
[ INFO  ] vmifucalib:     277 out of 278 ex

In [87]:
mscreenflat = 'ifu_master_screen_flat.fits'
arcextracted = 'ifu_arc_spectrum_extracted.fits'
flatextracted ='ifu_flat_spectrum_extracted.fits'
trace = 'ifu_trace.fits'
transmission = 'ifu_transmission.fits'
ids = 'ifu_ids.fits'
qc0='qc0000.paf'
qc1='qc0001.paf'
qc2 = 'qc0002.paf'
qc3='qc0003.paf'
products = [mscreenflat,arcextracted,flatextracted,trace,transmission,ids,qc0
           ,qc1,qc2,qc3]

for p in products:
    copycommand = "mv {} {}".format(p,productfolder)
    os.system(copycommand)

In [88]:
! ls {productfolder}

ifu_arc_spectrum_extracted.fits   ifu_trace.fits	 qc0001.paf
ifu_flat_spectrum_extracted.fits  ifu_transmission.fits  qc0002.paf
ifu_ids.fits			  master_bias.fits	 qc0003.paf
ifu_master_screen_flat.fits	  qc0000.paf


# Standard

In [89]:
specphot = 'ifu_specphot_table.fits'


fileids = [(productfolder+'/'+ids,'IFU_IDS')]
filetrace = [(productfolder+'/'+trace,'IFU_TRACE')]
filetrans = [(productfolder+'/'+transmission,'IFU_TRANSMISSION')]
filespecphot = [(productfolder+'/'+specphot,'IFU_SPECPHOT_TABLE')]
inputfiles = [masterbias,fileids,filetrace,filetrans,stdlist[0:1],stdtable[0:1],extable]
inputfiles = list(flatten(inputfiles))


filenamestandard = datadir+'ifustandard.sof'

with open(filenamestandard,'w') as file:
    for sof in inputfiles:
        string = '{} {}\n'.format(sof[0],sof[1])
        file.write(string)

In [90]:
! cat {filenamestandard}
routine = 'vmifustandard'

../data/grism3/products/master_bias.fits MASTER_BIAS
../data/grism3/products/ifu_ids.fits IFU_IDS
../data/grism3/products/ifu_trace.fits IFU_TRACE
../data/grism3/products/ifu_transmission.fits IFU_TRANSMISSION
../data/grism1/STD/VIMOS.2003-06-03T09:44:10.439.fits IFU_STANDARD
../esorex/calibrations/calib/vimos-3.2.3/cal/g158_100.tfits STD_FLUX_TABLE
../esorex/calibrations/calib/vimos-3.2.3/cal/extinct_table.tfits EXTINCT_TABLE


In [91]:
%%bash -s "{esorexpath}" "{routine}"  "{filenamestandard}"
$1 $2 $3 


     ***** ESO Recipe Execution Tool, version 3.13.1  *****

[ INFO  ] vmifustandard: Loading input frames...
[ INFO  ] vmifustandard: Bias removal...
[ INFO  ] vmifustandard: Processing data from quadrant 1, HR_blue/Free, shutter ON...
[ INFO  ] vmifustandard: Processing data from quadrant 1...
[ INFO  ] vmifustandard: Processing spectra in pseudo-slit 2:
[ INFO  ] vmifustandard:   Load flat field tracing table...
[ INFO  ] vmifustandard:   Detect traceable spectra on standard star exposure...
[ INFO  ] vmifustandard:     36 spectra detected on standard star exposure.
[ INFO  ] vmifustandard:   Tracing detected spectra...
[ INFO  ] vmifustandard:   Identify traced spectra on standard star exposure...
[ INFO  ] vmifustandard:   Align flat field tracings to standard star exposure...
[ INFO  ] vmifustandard:   Simple extraction of standard star and sky spectra...
[ INFO  ] vmifustandard:   Align wavelength calibration to detected sky lines...
[ INFO  ] vmifustandard:   Applying median s

In [92]:
ls

bias02download.sh                ifu_standard_reduced.fits
downloadRequest357281script.sh*  ifu_std_fov.fits
esorex.log                       qc0000.paf
extractZfiles.ipynb              quadranone.ipynb
extractZfiles.py*                quadranthree.ipynb
ifu_science_sky.fits             quadrantwo.ipynb
ifu_specphot_table.fits          seetypes.ipynb
ifu_standard_extracted.fits      stdjune2.sh


In [93]:
specphot = 'ifu_specphot_table.fits'
standardext = 'ifu_standard_extracted.fits'
standardreduced ='ifu_standard_reduced.fits'
stdfov = 'ifu_std_fov.fits'
qc0='qc0000.paf'
qc1='qc0001.paf'
qc2 = 'qc0002.paf'
qc3='qc0003.paf'
ifu_science_sky = 'ifu_science_sky.fits'
products = [specphot,standardext,standardreduced,stdfov,qc0
           ,qc1,qc2,qc3,ifu_science_sky]

for p in products:
    copycommand = "mv {} {}".format(p,productfolder)
    os.system(copycommand)

In [94]:
ls {productfolder}

ifu_arc_spectrum_extracted.fits   ifu_standard_extracted.fits  qc0000.paf
ifu_flat_spectrum_extracted.fits  ifu_standard_reduced.fits    qc0001.paf
ifu_ids.fits                      ifu_std_fov.fits             qc0002.paf
ifu_master_screen_flat.fits       ifu_trace.fits               qc0003.paf
ifu_science_sky.fits              ifu_transmission.fits
ifu_specphot_table.fits           master_bias.fits


# The scientific data reduction process:

## Create a science sof for each one:

In [95]:
allobjectsfiles = glob.glob(objectfolder+'/*/VIMOS.2003*.fits')

fileids = [(productfolder+'/'+ids,'IFU_IDS')]
filetrace = [(productfolder+'/'+trace,'IFU_TRACE')]
filetrans = [(productfolder+'/'+transmission,'IFU_TRANSMISSION')]
filespecphot = [(productfolder+'/'+specphot,'IFU_SPECPHOT_TABLE')]

arcextractedfile = [(productfolder+'/'+arcextracted,"IFU_ARC_SPECTRUM_EXTRACTED")]
flatextractedfile =[(productfolder+'/'+flatextracted,'IFU_FLAT_SPECTRUM_EXTRACTED')]
standardreducedfile =[(productfolder+'/'+standardreduced,'IFU_STANDARD_REDUCED')]



calibrate = '--CalibrateFlux=true'

for ob in allobjectsfiles:
    #print(ob)
    obj = [(ob,'IFU_SCIENCE')]
    inputfiles = [masterbias,obj,fileids,filetrace,filetrans,extable,filespecphot]
    inputfiles = list(flatten(inputfiles))
    filedir = ob.split('VIMOS')[0]
    filenamescience = filedir+'ifuscience.sof'
    filenamebash = filedir+'reduced.sh'
    filenamecube = filedir+'cube.sof'
    outputdir = '--output-dir={}'.format(filedir)
    logdir = '--log-dir={}'.format(filedir)
    with open(filenamescience,'w') as file:
        for sof in inputfiles:
            string = '{} {}\n'.format(sof[0],sof[1])
            file.write(string)
    routine = 'vmifuscience'
    command = '{} {} {} {} {} {}'.format(esorexpath,outputdir,logdir,routine, calibrate, filenamescience)
    #print(command)
    os.system(command)
    #Cube
    ifuscienceredfile = [(filedir+'ifu_science_reduced.fits','IFU_SCIENCE_REDUCED')]
    ifusciencefluxeduced =  [(filedir+'ifu_science_flux_reduced.fits','IFU_SCIENCE_FLUX_REDUCED')]

    inputfilescube = [ifuscienceredfile,ifusciencefluxeduced,arcextractedfile,flatextractedfile,
                     standardreducedfile]
    inputfilescube = [ifuscienceredfile]
    inputfilescube = list(flatten(inputfilescube))
    with open(filenamecube,'w') as file:
        for sof in inputfilescube:
            string = '{} {}\n'.format(sof[0],sof[1])
            file.write(string)
    routine = 'vmifucombinecube'
    commandcube = '{} {} {} {}  {}'.format(esorexpath,outputdir,logdir,routine,  filenamecube)
    os.system(commandcube)

    

# Create Cube in each one
        Input                         Output

          IFU_ARC_SPECTRUM_EXTRACTED    IFU_ARC_SPECTRUM_CUBE
          IFU_FLAT_SPECTRUM_EXTRACTED   IFU_FLAT_SPECTRUM_CUBE
          IFU_SCIENCE_REDUCED           IFU_SCIENCE_CUBE
          IFU_SCIENCE_FLUX_REDUCED      IFU_SCIENCE_FLUX_CUBE
          IFU_STANDARD_REDUCED          IFU_STANDARD_CUBE


# Old obsolete

%%bash -s "{esorexpath}" "{routine}"  "{filenamescience}"
$1 $2 $3 

sky_trace = 'ifu_sky_trace.fits'
fov = 'ifu_fov.fits'
sciencered ='ifu_science_reduced.fits'
skyids = 'ifu_sky_ids.fits'

products = [sky_trace,fov,sciencered,skyids]

for p in products:
    if os.path.exists(productfolder+'/'+fov) and os.path.exists(productfolder+'/'+sciencered):
        
    copycommand = "mv {} {}".format(p,productfolder)
    os.system(copycommand)

os.path.exists(productfolder+'/'+fov)

productfolder

# Only one?

In [61]:
ls

bias02download.sh                extractZfiles.py*   seetypes.ipynb
downloadRequest357281script.sh*  quadranone.ipynb    stdjune2.sh
esorex.log                       quadranthree.ipynb
extractZfiles.ipynb              quadrantwo.ipynb


In [62]:
tryspec = fits.open('ifu_science_reduced.fits')
spec=tryspec[0]
spec.data.shape

FileNotFoundError: [Errno 2] No such file or directory: 'ifu_science_reduced.fits'

In [ ]:
tryfov = fits.open('ifu_fov.fits')
fov=tryspec[0]
#.data.shape

In [ ]:
fov.data.shape

In [ ]:
# 

## CHeck with those sky lines to see if the wavelength maatch it. 

Bright one was Oxygen [I]  5577

Sodium I 5889

H beta: 4860
    
He II : 4686